# ARMA baseline model


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [2]:
# load data from drive
df = pd.read_csv("../src/data/processed/CPI.csv", index_col='date')
df_first_diff = np.log(df).diff()
df = df_first_diff.dropna()

In [3]:
# stationarity test
from statsmodels.tsa.stattools import adfuller
def adfuller_test(df, col, print_results=False):
    # extracting only the passengers count using values function
    values = df[col]

    # passing the extracted passengers count to adfuller function.
    # result of adfuller function is stored in a res variable
    res = adfuller(values)
    if print_results:
        print("Augmented Dickey Fuller for " + col)

        # Printing the statistical result of the adfuller test
        print('Augmneted Dickey_fuller Statistic: %f' % res[0])
        print('p-value: %f' % res[1])

        # printing the critical values at different alpha levels.
        print('critical values at different levels:')
        for k, v in res[4].items():
            print('\t%s: %.3f' % (k, v))
    else:
        return res

In [4]:
results = []
df_first_diff.dropna(inplace=True)
for name in df_first_diff.columns:
    res = adfuller_test(df_first_diff, name, print_results=False)
    results.append([name, res[1]])

results = pd.DataFrame(results, columns=['series', 'p-value'])
results

,series,p-value
0,CUSR0000SEFA01,4.825228e-07
1,CUSR0000SEFB04,4.949368e-05
2,CUSR0000SEFC01,3.200328e-30
3,CUSR0000SEFD02,1.013153e-08
4,CUSR0000SEFD03,0.000000e+00
5,CUSR0000SEFJ02,6.035928e-08
6,CUSR0000SEFK01,2.052638e-11
7,CUSR0000SEFK02,4.454782e-07
8,CUSR0000SEFL01,9.991575e-16
9,CUSR0000SEFL03,5.010693e-22


In [5]:
# arima model
import statsmodels.api as sm
train = df_first_diff.iloc[:-24]
test = df_first_diff.iloc[-24:]

In [8]:
%%capture
import warnings
warnings.filterwarnings("ignore")  ## this suppresses warnings for this cell

# hyperparameter tuning to p and q values chosen via AIC
results = pd.DataFrame(columns=['p', 'q', 'AIC'])

for p in range(1, 4):
    for q in range(1, 4):
        model = sm.tsa.VARMAX(train, order=(p, q))
        result = model.fit(method='bfgs', maxiter=1000, disp=False)
        df = pd.DataFrame([{'p': p, 'q': q, 'AIC': result.aic}])
        results = pd.concat([results, df], ignore_index=True)
        
res_df = pd.DataFrame(results, columns=['p', 'q', 'AIC'])
res_df.sort_values(by=['AIC']).head()

In [9]:
print("The best parameters were" +  str(res_df.iloc[0]))
p = res_df.iloc[0].p
q = res_df.iloc[0].q
#model = sm.tsa.VARMAX(train, order=(1, 1))
#fit_res = model.fit(method='bfgs', disp=False)
#fit_res.summary()

The best parameters werep                 1
q                 1
AIC   -34936.814375
Name: 0, dtype: object


In [10]:
n_forecast = len(test)
forecast_df = fit_res.forecast(steps=n_forecast)
forecast_df.index = test.index

#pred_res = fit_res.get_forecast(steps=n_forecast)
#forecast_df = pred_res.predicted_mean
#conf_int = pred_res.conf_int()

NameError: name 'fit_res' is not defined

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np

mse = mean_squared_error(test, forecast_df)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(test, forecast_df)
print("rmse:", rmse)
print("mape:", mape)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10,4))
test.plot(ax=ax, label="Actual")
forecast_df.plot(ax=ax, label="Forecast", linestyle="--")
ax.legend()
plt.show()

In [ ]:
def plot_predictions(df_predict, df_true, title='code'):
    ncols = 3
    nrows = len(df_predict.columns)//3 + 1
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10,10))
    axes = axes.flatten() # easier indexing
    i = 0
    for name in df_predict.columns:
        data_predict = df_predict[name].dropna()
        data_true = df_true[name].dropna()
        if title == 'description':
            fig_name = var_dict[name]
        elif title == 'code':
            fig_name = name
        data_predict.plot(figsize=(10,10), ax = axes[i], title=fig_name)
        data_true.plot(figsize=(10,10), ax = axes[i])
        plt.xlabel('Date')
        plt.ylabel('CPI')
        i+=1
    plt.tight_layout(h_pad = 3)
    plt.show()

In [ ]:
plot_predictions(forecast_df, test)